**Original author:  Camilo J. Torres             
created:  march of 2024
          personal email: <camilojtorresc@gmail.com>
          email:  <camilo.torres@cern.ch>**

Version 8 updates:
- Implement a form to convert the ROOT5 files provided by CORSIKA into a Pandas data frame to analize the simulations with the method implemented in previous versions. 

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import os

In [2]:
# install and import uproot
!pip install uproot3
import uproot3 as uproot

Defaulting to user installation because normal site-packages is not writeable


In [3]:
#Path of the file with the data of particles
path = r'/home/user/Documents/BUAP/Estancia/CosmicRayReco/MCAnalysis/ClusterAnalysis/Data'
os.chdir(path)

In [4]:
# Data with ROOT5 format
root_data = 'DataUSC/DAT000004.root'

# Open the sim tree
simulation = uproot.open(root_data)['sim']

In [5]:
# Show the Branches in the 'sim' tree
simulation.keys()

[b'shower.', b'particle.', b'long.', b'cherenkov.']

In [6]:
simulation["shower."].keys()

[b'shower.TObject',
 b'shower.fParticles',
 b'shower.fCherenkov',
 b'shower.EventID',
 b'shower.Energy',
 b'shower.StartingAltitude',
 b'shower.FirstTarget',
 b'shower.FirstHeight',
 b'shower.Theta',
 b'shower.Phi',
 b'shower.RandomSeed[10]',
 b'shower.RandomOffset[10]',
 b'shower.nPhotons',
 b'shower.nElectrons',
 b'shower.nHadrons',
 b'shower.nMuons',
 b'shower.nParticlesWritten',
 b'shower.nPhotonsWritten',
 b'shower.nElectronsWritten',
 b'shower.nHadronsWritten',
 b'shower.nMuonsWritten',
 b'shower.GH_Nmax',
 b'shower.GH_t0',
 b'shower.GH_tmax',
 b'shower.GH_a',
 b'shower.GH_b',
 b'shower.GH_c',
 b'shower.GH_Chi2',
 b'shower.nPreshower',
 b'shower.CPUtime']

In [7]:
ShowerVar = ['shower.EventID','shower.Energy','shower.FirstHeight',
             'shower.Theta','shower.Phi','shower.nParticlesWritten'] # Choose the interest variables

Showers = simulation.arrays(ShowerVar, outputtype=pd.DataFrame) # Transform into pdf
Showers = Showers[Showers['shower.nParticlesWritten']!=0] # Ignore showers without particles in measurement height

# Redefine the variables in the pdf
Showers['shower.FirstHeight'] = Showers['shower.FirstHeight']/1e5
Showers['shower.Theta'] = Showers['shower.Theta']*180/np.pi
Showers['shower.Phi'] = Showers['shower.Phi']*180/np.pi

# Rename columns
Showers = Showers.rename(columns={ShowerVar[0]: 'NShow', ShowerVar[1]: 'Energy',
                                  ShowerVar[2]: 'ZFInt', ShowerVar[3]: 'ZhAng',
                                  ShowerVar[4]: 'AzAng', ShowerVar[5]: 'NParticles'})

Showers

,NShow,Energy,ZFInt,ZhAng,AzAng,NParticles
entry,,,,,,
4,5,7.579618,15.897397,30.570698,-104.965866,1
9,10,7.955384,10.797235,33.753933,165.915329,1
13,14,10.202897,15.586424,11.866905,17.410604,2
22,23,11.754333,13.532733,27.440584,-60.561756,1
44,45,8.494957,24.328083,7.733621,36.078545,1
...,...,...,...,...,...,...
99978,99979,8.870023,23.504463,11.639350,-99.643448,1
99979,99980,10.474447,14.565544,12.646334,-18.351109,2
99988,99989,8.313620,11.972643,26.025732,106.134659,1


In [ ]:
simulation["particle."].keys()

In [ ]:
# It is a bit more complex for particles to extract into a Pandas data frame

ParticleVar = ['shower.EventID','shower.nParticlesWritten','particle..ParticleID',
               'particle..x','particle..y','particle..Time',
              'particle..Px','particle..Py','particle..Pz'] # Interest variables

particle_df = simulation.arrays(ParticleVar, outputtype=pd.DataFrame) # Extract the variables from branch
particle_df = particle_df[particle_df["shower.nParticlesWritten"]!=0] # Ignore showers without particles in measurement height

partnum = particle_df.size # Number of particles 

particle_df = particle_df.to_dict() # Transform into dictionary

In [ ]:
%%time
Particles = {"NShow": [], "PId": [], "X": [], "Y": [], "T": [],
            "Px":[], "Py": [], "Pz": []} # Create the new data frma
Particles = pd.DataFrame(Particles) 

ShowerNum = Showers['NShow'].tolist() # List with all the shower id with particles at measurement height

for i in range(len(ShowerNum)):
    part_sh = particle_df['particle..ParticleID'][ShowerNum[i]-1].size # Number of particles in the shower
    
    for j in range(part_sh): # Fill the Particles pdf with the info of the dictionary 
        list_for_df = [ShowerNum[i],
                      particle_df['particle..ParticleID'][ShowerNum[i]-1][j],
                      particle_df['particle..x'][ShowerNum[i]-1][j],
                      particle_df['particle..y'][ShowerNum[i]-1][j],
                      particle_df['particle..Time'][ShowerNum[i]-1][j],
                      particle_df['particle..Px'][ShowerNum[i]-1][j],
                      particle_df['particle..Py'][ShowerNum[i]-1][j],
                      particle_df['particle..Pz'][ShowerNum[i]-1][j]
                      ]

        Particles.loc[i+1] = list_for_df # Fill new row
        
# Redefine the variables in the pdf
Particles["X"] = Particles["X"]/1e2
Particles["Y"] = Particles["Y"]/1e2
Particles["PSq"] = Particles["Px"]**2+Particles["Py"]**2+Particles["Pz"]**2
Particles["Ene"] = np.sqrt(Particles["PSq"])
Particles["ZhA"] = np.arctan2(np.sqrt((Particles["Py"])**2+(Particles["Px"])**2),Particles["Pz"])*(180.0/np.pi)
Particles["AzA"] = np.arctan2(Particles["Py"],Particles["Px"])*(180.0/np.pi)

Particles

In [ ]:
#File with summary of Showers
#Showers = pd.read_csv(r'DAT000006_1k_showers.txt',delimiter='\t')

In [ ]:
#File with information of all particles at observation level
#Particles = pd.read_csv('DAT000006_1k_particles.txt',delimiter='\t')

In [ ]:
# Run summary
Energies = {'E1': 9, 'E2': 11, 'E3': 14, 'E4': 18, 'E5': 55, 'E6':70}
Run = {'SCod': 'Corsika-77500','Mass':[1],'Lab': ['BUAP'], 'NShows':[100000], 'EInf': [7] , 'ESup': [13], 'AAInf':[0], 'AASup':[37]} 
Run = pd.DataFrame(Run)

Run

In [ ]:
Showers['Mass'] = 1

In [ ]:
#Statistics of the data frame with summary of showers 
Showers.describe()

In [ ]:
# Define local particles codes for labeling clusters
# Particle Corsika PID       P. Code
# Gamma        1              1
# Electrons    2,3            1000
# Muons        5,6            100000
# Pions        8,9            50000000
# Protons      14             10000000
# Neutrons     13             100000000
nop   = -999.
gam   = 1.
ele   = 1000.
mu    = 100000.
pi    = 50000000.
prn   = 10000000.
ntn   = 100000000. 

pmap   = {1.:gam,2.:ele,3.:ele,5.:mu,6.:mu,8.:pi, 9.:pi, 13.:ntn, 14.:prn}
#pmap   = {'1.':'gam','2.':'ele','3.':'ele','5.':'mu','6.':'mu','8.':'pi','9.':'pi', '13.':'ntn', '14.':'prn'}
#pid    = [1., 2., 3., 5., 6., 8., 9., 13., 14.]
#zipped = list(zip(pid,pcode))
#pcode  = pd.DataFrame(zipped, columns=['pid', 'pcode'])
Particles['R'] = np.sqrt(Particles['X']**2 + Particles['Y']**2)
Particles['PCode'] = Particles['PId'].map(pmap)

In [ ]:
Particles.describe()

In [ ]:
# Observatory layout
det_s_x = 1.0 # detector size in x axis (m)
det_s_y = 1.0 # detector size in y axis (m)

In [ ]:
#The values in the 'DetMdx' and 'DetMdy' columns will be replaced by the rounded values 
#of the positions of the particles, this will define the position of the cell
#
Particles['DetX'] = np.floor(Particles['X']/det_s_x) + det_s_x/2
Particles['DetY'] = np.floor(Particles['Y']/det_s_x) + det_s_y/2
#
Particles['PartX'] = 1000 * (Particles['X'] - Particles['DetX']) # mm
Particles['PartY'] = 1000 * (Particles['Y'] - Particles['DetY']) # mm
#Add a new column which will contain the number of cluster 
Particles['NClst'] = 0

In [ ]:
Particles.head()

In [ ]:
def Clusters(iclust, PartS, ClustDF):
    # Find clusters of particles in a shower
    # iclust: number of cluster
    # Shower: number of shower
    
    XPos = PartS['DetX'].tolist()
    YPos = PartS['DetY'].tolist()
    
    unique_pairs = set() # Create an empty set

    # Iterate over the pairs of x and y using zip
    for xi, yi in zip(XPos, YPos):
        pair = (xi, yi)
        
        #VarClust = []

        # Check if the pair is already in the set
        if pair not in unique_pairs:
            unique_pairs.add(pair) # If not, add it to the set

            # Extract the particles index by comparing it with the x, y position in the data frame
            p_index_clust = PartS.index[ (PartS['DetX'] == xi) & (PartS['DetY'] == yi) ].tolist()
            
            # Replacte the cluster position of the particle for the cluster id
            for j in range(len(p_index_clust)):
                PartS.at[p_index_clust[j],'NClst'] = iclust
            
            # For one particle clusters
            if(len(p_index_clust) == 1):
                
                df_clust = PartS[PartS['NClst']==iclust]
                
                ClsIdC = df_clust["PCode"].tolist()[0]
                NpartC = df_clust.shape[0]
                NGamC = df_clust[df_clust['PCode']==1.].shape[0]
                NEleC = df_clust[df_clust['PCode']==1000.].shape[0]
                NMuCl = df_clust[df_clust['PCode']==100000.].shape[0]
                XmClst = df_clust["X"].tolist()[0]
                YmClst = df_clust["Y"].tolist()[0]
                RmClst = df_clust["R"].tolist()[0]
                SigRCl = 0
                TmClst = df_clust["T"].tolist()[0]
                dTClst = df_clust["T"].max()-df_clust["T"].min()
                sTClst = 0
                FstPID = ClsIdC
                FstPCX = df_clust["Px"].tolist()[0]
                FstPCY = df_clust["Py"].tolist()[0]
                FstPCT = df_clust["T"].tolist()[0]
                FstPZh = df_clust["ZhA"].tolist()[0]
                FstPAz = df_clust["AzA"].tolist()[0]
                FstPPm = np.sqrt(FstPCX**2+FstPCY**2+df_clust["Pz"].tolist()[0]**2)
                LstPID = FstPID
                LstPCX = FstPCX
                LstPCY = FstPCY
                LstPCT = FstPCT
                LstPZh = FstPZh
                LstPAz = FstPAz
                LstPPm = FstPPm
            
            # For clusters with more than 1 particle
            if(len(p_index_clust) > 1):
                # Calculate variables and fill the Cluster dataframe
                df_clust = PartS[PartS['NClst']==iclust]
                FirstPart = df_clust[df_clust['T'] == df_clust["T"].min()]
                LastPart = df_clust[df_clust['T'] == df_clust["T"].max()]
                
                ClustStats_mean = df_clust[["X","Y","R","T"]].mean()
                ClustStats_std  = df_clust[["X","Y","R", "T"]].std()
                
                ClsIdC = df_clust["PCode"].sum()
                NpartC = df_clust.shape[0]
                NGamC = df_clust[df_clust['PCode']==1.].shape[0]
                NEleC = df_clust[df_clust['PCode']==1000.].shape[0]
                NMuCl = df_clust[df_clust['PCode']==100000.].shape[0]
                XmClst = ClustStats_mean["X"]
                YmClst = ClustStats_mean["Y"]
                RmClst = ClustStats_mean["R"]
                SigRCl = ClustStats_std["R"]
                TmClst = ClustStats_mean["T"]
                dTClst = df_clust["T"].max()-df_clust["T"].min()
                sTClst = ClustStats_std["T"]
                FstPID = FirstPart["PCode"].tolist()[0]
                FstPCX = FirstPart["Px"].tolist()[0]
                FstPCY = FirstPart["Py"].tolist()[0]
                FstPCT = FirstPart["T"].tolist()[0]
                FstPZh = FirstPart["ZhA"].tolist()[0]
                FstPAz = FirstPart["AzA"].tolist()[0]
                FstPPm = np.sqrt(FirstPart["Px"].tolist()[0]**2+FirstPart["Py"].tolist()[0]**2+FirstPart["Pz"].tolist()[0]**2)
                LstPID = LastPart["PCode"].tolist()[0]
                LstPCX = LastPart["Px"].tolist()[0]
                LstPCY = LastPart["Py"].tolist()[0]
                LstPCT = LastPart["T"].tolist()[0]
                LstPZh = LastPart["ZhA"].tolist()[0]
                LstPAz = LastPart["AzA"].tolist()[0]
                LstPPm = np.sqrt(LastPart["Px"].tolist()[0]**2+LastPart["Py"].tolist()[0]**2+LastPart["Pz"].tolist()[0]**2)
                

            VarClust = [iclust, ClsIdC, NpartC, NGamC, NEleC, NMuCl,
                           XmClst, YmClst, RmClst, SigRCl,
                           TmClst, dTClst, sTClst,
                           FstPID, FstPCX, FstPCY, FstPCT, FstPZh, FstPAz, FstPPm,
                           LstPID, LstPCX, LstPCY, LstPCT, LstPZh, LstPAz, LstPPm]
                
            ClustDF.loc[len(ClustDF.index)] = VarClust # Fill new row

            
            iclust = iclust+1 # Next Cluster
    
    return iclust, PartS, ClustDF

In [ ]:
%%time
# Define the dataframe for cluster info.
ClusterDF = {"NClst": [],'ClsIdC': [], 'NpartC': [], "NGamC": [], "NEleC": [],"NMuCl": [],
            "XmClst": [], "YmClst": [], "RmClst": [], "SigRCl": [],
            "TmClst": [], "dTClst": [], "sTClst": [],
            "FstPID": [], "FstPCX": [], "FstPCY": [], "FstPCT": [], "FstPZh": [], "FstPAz": [], "FstPPm": [],
            "LstPID": [], "LstPCX": [], "LstPCY": [], "LstPCT": [], "LstPZh": [], "LstPAz": [], "LstPPm": []
            } 

ClusterDF = pd.DataFrame(ClusterDF)
iclust = 1 # Cluster counter

# Run over all the showers
nsh = Showers['NShow'].tolist()
for ishow in nsh:
    Shower = Particles[Particles['NShow']==ishow]   #current shower
    iclust, Nshow, ClusterDF = Clusters(iclust, Shower, ClusterDF) 
    Particles.loc[Particles['NShow']==ishow, :] = Nshow

In [ ]:
# Check if there are particles that are not counted
Particles[Particles["NClst"]==0]

In [ ]:
ClusterDF

In [ ]:
# Number of the cluster with the maximum number of particles
df_sn = Particles[Particles["NClst"]!=0]
Max_ClustId = df_sn["NClst"].value_counts().idxmax()
print(Max_ClustId)

In [ ]:
# Summary of the cluster with the maximum number of particles
Particles[Particles["NClst"]==Max_ClustId]

In [ ]:
# Summary of the cluster with the maximum number of particles
ClusterDF[ClusterDF["NClst"]==Max_ClustId]

In [ ]:
#Plot the cell with the maximum number of particles
cx = Particles.loc[Particles["NClst"]==Max_ClustId,'DetX'].iloc[0]
cy = Particles.loc[Particles["NClst"]==Max_ClustId,'DetY'].iloc[0]
Mx = Particles.loc[Particles["NClst"]==Max_ClustId,'PartX']
My = Particles.loc[Particles["NClst"]==Max_ClustId,'PartY']
plt.scatter(Mx,My)
plt.title('Cell: X=%i, Y=%i'%(cx,cy))
plt.xlabel('X (mm)')
plt.ylabel('Y (mm)')
#Cell for detector of 1 m^2 
plt.axis([-500,500,-500,500])
#Cell for detector of 0.30 x 0.30 m^2
#plt.axis([-150,150,-150,150])
plt.grid(True)